### Task
* Given
    * Operating temp & pressure (K, atm)
    * Reaction Kinetics Information
    * Reaction Thermochemistry Information (MJ/kmol)
    * Reaction Stoichiometry
    * Composition at inlet (kmol/m3)
<br>
* To find
    * Composition at outlet (kmol/m3)
    * Cooling/Heating duty (MJ) if reactor is isothermal

In [2]:
import pandas as pd
import math
from typing import Callable
import pfr_utils as pu

In [2]:
molecular_wt = (25,50,25,25)
sp_heat_cap = (1.1,0.8,1.5,0.9)     # kJ/kg-K
heat_of_rxn = -50000                # kJ/kmol NOTE: Exothermic reaction with no
        # cooling => Temp increase inhibits reaction.
cooling = False
cooling_duty_kJ = 0        # kJ. Initialise

t_points = [float(f"{(i*math.pi/10):.2f}") for i in range(10)]

#### 1.1. Steady-State Simulation using Euler's finite difference method
* 2A + B = C + 3D

In [ ]:
moles_A = {0.00:0.8} # Keys will be z-co-ordinate
moles_B = {0.00:0.5}
moles_C = {0.00:0}
moles_D = {0.00:0}

Temp_kelvin = {0.00:900}

if cooling is True:
    rate_const_entry = rate_const(Temp_kelvin[list(Temp_kelvin.keys())[0]])

# For 10 nodes
for i in range(10):
    z_coord = float(f"{(i*length_step):.2f}")
    z_next = float(f"{(z_coord + length_step):.2f}")
    
    # Calculate rate of reaction in this element w.r.t. A using temperature at 
    # entry of elemental volume
    if cooling is False:
        rate_A = rate_const(Temp_kelvin[z_coord]) * (moles_A[z_coord] ** abs(stoic_cf[0]))
    else:
        rate_A = rate_const_entry * (moles_A[z_coord] ** abs(stoic_cf[0]))
    
    # Calculate concentration to check for limiting reactant
    moles_reacted_A = (rate_A * (length_step ** 2)/(axial_vel))
    next_moles_A = moles_A[z_coord] - moles_reacted_A

    next_moles_B = moles_B[z_coord] - \
                    (stoic_cf[1]/stoic_cf[0]) * moles_reacted_A
    
    next_moles_C = moles_C[z_coord] - \
                    (stoic_cf[2]/stoic_cf[0]) * moles_reacted_A
    
    next_moles_D = moles_D[z_coord] - \
                    (stoic_cf[3]/stoic_cf[0]) * moles_reacted_A

    # Check for limiting reactant. Only proceed if no concentration is equal to 0
    if next_moles_A > 0 and next_moles_B > 0:
        # Calculate moles for next node
        moles_A[z_next] = next_moles_A
        moles_B[z_next] = next_moles_B
        moles_C[z_next] = next_moles_C
        moles_D[z_next] = next_moles_D

        # Calculate temperature for next node & cooling duty as per settings of reactor cooling
        if cooling is False:
            
            # Calculate density & specific heat capacity
            moles_state = [
                moles_A[z_coord],moles_B[z_coord],
                moles_C[z_coord],moles_D[z_coord]
                ]
            element_mass = sum([a*b 
                                for a,b in zip(moles_state,molecular_wt)
                                ])
            element_sp_heat_cap = sum([
                a*b*c*1e-3 
                for a,b,c in zip(moles_state,sp_heat_cap,molecular_wt)
                ])/element_vol
            
            # Calculate temperature for next node
            Temp_kelvin[z_next] = Temp_kelvin[z_coord] + (moles_reacted_A*heat_of_rxn/\
                                                      (element_mass*element_sp_heat_cap))
            
        else:
            cooling_duty_kJ += moles_reacted_A*heat_of_rxn
            Temp_kelvin[z_next] = Temp_kelvin[z_coord]
    
    # Else if some concentration is zero
    else:
        # Determine moles and temperature for next node, as being the same as current node
        moles_A[z_next] = moles_A[z_coord]
        moles_B[z_next] = moles_B[z_coord]
        moles_C[z_next] = moles_C[z_coord]
        moles_D[z_next] = moles_D[z_coord]
        Temp_kelvin[z_next] = Temp_kelvin[z_coord]
        

In [ ]:
df_moles = pd.DataFrame(data=[moles_A,moles_B,moles_C,moles_D],
                       index=["moles_A","moles_B","moles_C","moles_D"]).transpose()
df_moles

In [ ]:
from matplotlib_funcs import *

plot_matplotlib_style(df_moles.index,df_moles["moles_A"],"Length","Moles of A (mol)","Moles of A (mol)")
plot_matplotlib_style(df_moles.index,df_moles["moles_B"],"Length","Moles of B (mol)","Moles of B (mol)")
plot_matplotlib_style(df_moles.index,df_moles["moles_C"],"Length","Moles of C (mol)","Moles of C (mol)")
plot_matplotlib_style(df_moles.index,df_moles["moles_D"],"Length","Moles of D (mol)","Moles of D (mol)")
plot_matplotlib_style(df_moles.index,Temp_kelvin.values(),"Length","Temperature (K)","Temperature (K)")

In [ ]:
cooling_duty_kJ

#### 1.2. Transient Simulation using Euler's Finite Difference Method
* Transient Mass Balance
* Transient Energy Balance
* [Tutorial: Numerical Solution of PDEs by FDM](https://math.libretexts.org/Bookshelves/Differential_Equations/Introduction_to_Partial_Differential_Equations_(Herman)/10%3A_Numerical_Solutions_of_PDEs/10.02%3A_The_Heat_Equation)

In [3]:
class SteadyState:
    solver : pu.ReactorFDMSolver
    reactor : pu.Reactor
    reaction : pu.Reaction

    @staticmethod
    def calc_conc_temp_profile(self,entry_mol:list,
        axial_vel:float,
        )->dict[str,dict[float,float]]:    
        
        moles_A = {0.00:entry_mol[0]} # Keys will be z-co-ordinate
        moles_B = {0.00:entry_mol[1]}
        moles_C = {0.00:entry_mol[2]}
        moles_D = {0.00:entry_mol[3]}
        Temp_kelvin = {0.00:900}
        element_vol = self.reactor.cross_section_area * self.solver.length_step

        if cooling is True:
            rate_const_entry = self.reaction.rate_const(Temp_kelvin[list(Temp_kelvin.keys())[0]])

        # For 10 nodes
        for i in range(10):
            z_coord = float(f"{(i*self.solver.length_step):.2f}")
            z_next = float(f"{(z_coord + self.solver.length_step):.2f}")
            
            # Calculate rate of reaction in this element w.r.t. A using temperature at 
            # entry of elemental volume
            if cooling is False:
                rate_A = self.reaction.rate_const(Temp_kelvin[z_coord]) * (moles_A[z_coord] ** abs(self.reaction.stoic_cf["A"]))
            else:
                rate_A = rate_const_entry * (moles_A[z_coord] ** abs(self.reaction.stoic_cf["A"]))
            
            # Calculate concentration to check for limiting reactant
            moles_reacted_A = (rate_A * (self.solver.length_step ** 2)/(axial_vel))
            next_moles_A = moles_A[z_coord] - moles_reacted_A

            next_moles_B = moles_B[z_coord] - \
                            (self.reaction.stoic_cf["B"]/self.reaction.stoic_cf["A"]) * moles_reacted_A
            
            next_moles_C = moles_C[z_coord] - \
                            (self.reaction.stoic_cf["C"]/self.reaction.stoic_cf["A"]) * moles_reacted_A
            
            next_moles_D = moles_D[z_coord] - \
                            (self.reaction.stoic_cf["D"]/self.reaction.stoic_cf["A"]) * moles_reacted_A

            # Check for limiting reactant. Only proceed if no concentration is equal to 0
            if next_moles_A > 0 and next_moles_B > 0:
                # Calculate moles for next node
                moles_A[z_next] = next_moles_A
                moles_B[z_next] = next_moles_B
                moles_C[z_next] = next_moles_C
                moles_D[z_next] = next_moles_D

                # Calculate temperature for next node & cooling duty as per settings of reactor cooling
                if cooling is False:
                    
                    # Calculate density & specific heat capacity
                    moles_state = [
                        moles_A[z_coord],moles_B[z_coord],
                        moles_C[z_coord],moles_D[z_coord]
                        ]
                    element_mass = sum([a*b 
                                        for a,b in zip(moles_state,molecular_wt)
                                        ])
                    element_sp_heat_cap = sum([
                        a*b*c*1e-3 
                        for a,b,c in zip(moles_state,sp_heat_cap,molecular_wt)
                        ])/element_vol
                    
                    # Calculate temperature for next node
                    Temp_kelvin[z_next] = Temp_kelvin[z_coord] + (moles_reacted_A*heat_of_rxn/\
                                                            (element_mass*element_sp_heat_cap))
                    
                else:
                    cooling_duty_kJ += moles_reacted_A*heat_of_rxn
                    Temp_kelvin[z_next] = Temp_kelvin[z_coord]
            
            # Else if some concentration is zero
            else:
                # Determine moles and temperature for next node, as being the same as current node
                moles_A[z_next] = moles_A[z_coord]
                moles_B[z_next] = moles_B[z_coord]
                moles_C[z_next] = moles_C[z_coord]
                moles_D[z_next] = moles_D[z_coord]
                Temp_kelvin[z_next] = Temp_kelvin[z_coord]

        return {**dict.fromkeys(self.reaction.stoic_cf,[moles_A,moles_B,moles_C,moles_D]),
                 "Temperature (K)":Temp_kelvin}

In [3]:
my_reactor = pu.Reactor(0.4,2)
my_reaction = pu.Reaction((-2,-1,1,3),2e-3,5)
my_solver = pu.ReactorFDMSolver(1e-2,0.1)

In [8]:
class TransientState:
    """Namespace for concentration & temperature profile for dynamic simulation
    of PFR"""
    solver : pu.ReactorFDMSolver
    reactor : pu.Reactor
    reaction : pu.Reaction

    def calc_conc_temp_profile(self,time_pts:list):
        """Calculate unsteady concentrations and temperature profile for PFR."""
        
        # Define state variables: n_i, T
        state = {}
        
        # 1st for-loop for all time-points
        for t in time_pts:
            # Inner for-loop for all nodes/positions
            for i in range(10):
                # Determine species moles at entry of element
                entry_moles = [4,5,0,0.1]   # TODO: Should this be dynamic?
                axial_vel = {t:1.2 + 0.4*math.sin(2*t) for t in t_points}
                state[t] = SteadyState.calc_conc_temp_profile(entry_moles,axial_vel)

                # Core equation for transient energy balance
                heating_duty - shaft_work = sum([
                    specie[z_coord] * sp_heat_cap[idx] * \
                        ((Temp_kelvin[t][z_coord]-\
                          Temp_kelvin[t-self.solver.time_step][[z_coord]])/\
                            self.solver.time_step)
                    for idx, specie in enumerate(self.reaction.stoic_cf.keys())
                    ], rate_A * element_vol * heat_of_rxn)

# run function
my_transient_bal = TransientState()
my_transient_bal.reaction = my_reaction
my_transient_bal.reactor = my_reactor
my_transient_bal.solver = my_solver

# Arguments for SteadyState.calc_conc_temp_profile
entry_moles = [4,5,0,0.1]   # TODO: Should this be dynamic?